# Parcours Data scientist - Analysez des données nutritionnelles

In [1]:
# Récupération du fichier en local
import pandas as pd
import numpy as np
%pylab inline

dataraw = pd.read_csv('C:\Work\OpenClassrooms\data/fr.openfoodfacts.org.products.csv', sep="\t", low_memory=False)

print(dataraw.shape)
dataraw.head()

Populating the interactive namespace from numpy and matplotlib
(320772, 162)


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,0000000003087,http://world-fr.openfoodfacts.org/produit/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000004530,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.0,NaN,NaN
2,0000000004559,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Peanuts,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
3,0000000016087,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489055731,2017-03-09T10:35:31Z,1489055731,2017-03-09T10:35:31Z,Organic Salted Nut Mix,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,12.0,NaN,NaN
4,0000000016094,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489055653,2017-03-09T10:34:13Z,1489055653,2017-03-09T10:34:13Z,Organic Polenta,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# On remplace les '-' des noms de colonne par '_' pour pouvoir requêter
for col in dataraw.columns:
    if '-' in col:
        #print("rename %s in %s" % (col, col.replace('-', '_')))
        dataraw = dataraw.rename(columns= {col : col.replace('-', '_')})
#dataraw.head()

# Nettoyage

In [3]:
# On enlève les colonnes dont toutes les valeurs sont nulles
my_data = dataraw.dropna(axis=1, how='all')
print(my_data.shape)
print("%i colonnes en moins (valeurs toutes nulles)" % (dataraw.shape[1]-my_data.shape[1]))

# On enlève les colonnes avec trop peu d'information (1 pour 1000)
nb = 0
nbMax = (int)(my_data.shape[0] * 0.001)
for col in dataraw.columns:
    if my_data[col].count() < nbMax:
        #print("%s : %i" % (i, my_data[i].count()))
        my_data = my_data.drop(col, axis=1)
        nb += 1
print("%i colonnes enlevées (moins de %i valeurs)" % (nb, nbMax))
print(my_data.shape)

(320772, 146)
16 colonnes en moins (valeurs toutes nulles)
45 colonnes enlevées (moins de 320 valeurs)
(320772, 101)


## Autres colonnes

>On a encore beaucoup de colonnes dont certaines pourraient être utiles pour un client recherchant des aliments sains :
>>- Présence d'additifs

>>- Ingrédients contenant de l'huile de palme

>>- Origine, pays de provenance et empreinte carbone pour une recherche de circuit court...

>Ces colonnes demanderait un peu plus de temps d'analyse donc dans un premier temps on va se concentrer sur les valeurs numériques que l'on a et ne garder que les colonnes du type xxx_100g.

>On va se baser sur le score français pour la suite (on verra que le score uk et le grade sont des informations redondantes) 


In [45]:
# On ne garde que les colonnes avec des valeurs nutritionnelles et les scores
my_data_nut = my_data.copy()
for i in my_data_nut.columns:
    if not ('_100g' in i or 'nutrition' in i):
        #print(i)
        my_data_nut = my_data_nut.drop(i, axis=1)
print("%i colonnes en moins (colonnes sans valeurs nutritionnelles)" % (my_data.shape[1]-my_data_nut.shape[1]))
print(my_data_nut.shape)

# On enlève les lignes dont toutes les valeurs sont nulles
my_data_nut = my_data_nut.dropna(axis=0, how='all')
print("%i lignes en moins (valeurs toutes nulles)" % (my_data.shape[0]-my_data_nut.shape[0]))
print(my_data_nut.shape)

56 colonnes en moins (colonnes sans valeurs nutritionnelles)
(320772, 45)
55387 lignes en moins (valeurs toutes nulles)
(265385, 45)


## Valeurs nulles

In [46]:
# On met des zéros dans les valeurs nutritives nulles
for i in my_data_nut.columns:
    if not ('nutrition' in i):
        my_data_nut[i] = my_data_nut[i].fillna(0)


## Valeurs négatives

In [47]:
# Annulation des valeurs nutritives négatives s'il y en a
for i in my_data_nut.columns:
    if 'nutrition' in i: continue
    neg = my_data_nut[my_data_nut[i] < 0].shape[0]
    if neg > 0:
        print(neg, 'valeurs négatives pour ', i)
    # On remplace par 0
    my_data_nut[i] = my_data_nut[i].clip(lower=0)


4 valeurs négatives pour  trans_fat_100g
7 valeurs négatives pour  sugars_100g
1 valeurs négatives pour  fiber_100g
3 valeurs négatives pour  proteins_100g
1 valeurs négatives pour  vitamin_a_100g
1 valeurs négatives pour  vitamin_c_100g
8 valeurs négatives pour  biotin_100g
8 valeurs négatives pour  pantothenic_acid_100g
1 valeurs négatives pour  iron_100g
1 valeurs négatives pour  copper_100g
1 valeurs négatives pour  selenium_100g


## Outliers

In [48]:
print(my_data_nut.shape)

for i in my_data_nut.columns:
    if 'nutrition' in i:
        continue

    # Etude sur un échantillon représentatif (les valeurs positives)
    df = my_data_nut[my_data_nut[i]>0]
#    nbTot = my_data_nut[my_data_nut[i]>0].shape[0]
#    if nbTot < 5000:
#        print("Trop peu de valeurs (%i) pour %s" % (nbTot, i))
#        continue

    # Outliers
    q75, q25 = np.percentile(df[i], [75 ,25])
    iqr = q75 - q25

    #min = q25 - (iqr*1.5)
    #my_data_nut = my_data_nut[my_data_nut[i]>min]

    max = q75 + (iqr*100) # Je prends une valeur très haute car il sinon on sort trop de lignes
    nbOutliers = my_data_nut[my_data_nut[i]>max].shape[0]
    if nbOutliers > 0:
        print("Il y a %i valeurs supérieures à %f pour %s (max=%f)" % (nbOutliers, max, i, np.max(my_data_nut[i])))

    my_data_nut = my_data_nut[my_data_nut[i]<=max]
    
print(my_data_nut.shape)

(265385, 45)
Il y a 3 valeurs supérieures à 126074.000000 pour energy_100g (max=3251373.000000)
Il y a 1 valeurs supérieures à 312.570000 pour trans_fat_100g (max=369.000000)
Il y a 5 valeurs supérieures à 5.671000 pour cholesterol_100g (max=95.238000)
Il y a 46 valeurs supérieures à 138.684000 pour salt_100g (max=64312.800000)
Il y a 19 valeurs supérieures à 0.024856 pour vitamin_a_100g (max=26.700000)
Il y a 3 valeurs supérieures à 0.000231 pour vitamin_d_100g (max=0.030000)
Il y a 7 valeurs supérieures à 1.032000 pour vitamin_e_100g (max=15.100000)
Il y a 2 valeurs supérieures à 0.009593 pour vitamin_k_100g (max=31.250000)
Il y a 85 valeurs supérieures à 1.871400 pour vitamin_c_100g (max=716.981100)
Il y a 6 valeurs supérieures à 42.929000 pour vitamin_b1_100g (max=161.000000)
Il y a 8 valeurs supérieures à 30.357071 pour vitamin_b2_100g (max=42.500000)
Il y a 18 valeurs supérieures à 0.598191 pour vitamin_pp_100g (max=21.428571)
Il y a 22 valeurs supérieures à 0.124415 pour vitamin

## On exporte un fichier light pour une étude rapide

In [49]:
col_score = ['nutrition_score_fr_100g', 'nutrition_score_uk_100g', 'nutrition_grade_fr']

# On ne garde que les scores non nulls
dflight = my_data_nut.copy()
dflight.dropna(subset=col_score, inplace=True)
print(dflight.shape)
dflight.to_csv('clean_products_light.csv', sep="\t")

# On travaille aussi sur un échantillon pour la mémoire et le temps de calcul
dflight = dflight.sample(30000)
print(dflight.shape)
dflight.to_csv('clean_products_light_sample.csv', sep="\t")

(221017, 45)
(30000, 45)


## Recherche des scores manquants par regression knn

In [50]:
from sklearn import metrics, preprocessing, neighbors, model_selection

In [51]:
# On prend un échantillon pour des tests plus rapide
my_data_nut = my_data_nut.sample(10000)

In [52]:
# Séparation des valeurs nulles et non nulles
dfScoreNull = my_data_nut[pd.isnull(my_data_nut.nutrition_score_fr_100g)]
print('Il y a', dfScoreNull.shape[0], 'valeurs nulles')
dfScoreNotNull = my_data_nut[pd.notnull(my_data_nut.nutrition_score_fr_100g)]
print('Il y a', dfScoreNotNull.shape[0], 'valeurs non nulles')

# On sépare les colonnes de scores sur les valeurs non nulles
X = dfScoreNotNull.drop(col_score, axis=1)
y = dfScoreNotNull.nutrition_score_fr_100g

# 30% des données dans le jeu de test
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3)

# Standardisation
std_scale = preprocessing.StandardScaler().fit(X_train)
X_train_std = std_scale.transform(X_train)
X_test_std = std_scale.transform(X_test)

Il y a 1679 valeurs nulles
Il y a 8321 valeurs non nulles


In [53]:
# On fait un grid search pour trouver le meilleur k
param_grid = {'n_neighbors':[3, 5, 7, 9, 11, 13, 15]}
mse = 'neg_mean_squared_error'
gs = model_selection.GridSearchCV(neighbors.KNeighborsRegressor(), param_grid, scoring=mse, cv = 5)

gs.fit(X_train_std, y_train)
me = []
err = []
for mean, std, params in zip(gs.cv_results_['mean_test_score'], # score moyen
                             gs.cv_results_['std_test_score'], # écart-type du score
                             gs.cv_results_['params'] # valeur de l'hyperparamètre
                            ):
    print ("\t%s = %0.3f (+/-%0.03f) for %r" % ('MSE', -mean, std * 2, params))
    me.append(mean)
    err.append(2*std)

kBest = gs.best_params_['n_neighbors']
print ('Meilleur k : %r' % kBest)

	MSE = 12.417 (+/-3.011) for {'n_neighbors': 3}
	MSE = 11.953 (+/-2.361) for {'n_neighbors': 5}
	MSE = 12.157 (+/-2.471) for {'n_neighbors': 7}
	MSE = 12.374 (+/-2.430) for {'n_neighbors': 9}
	MSE = 12.514 (+/-2.200) for {'n_neighbors': 11}
	MSE = 12.735 (+/-2.114) for {'n_neighbors': 13}
	MSE = 12.944 (+/-2.070) for {'n_neighbors': 15}
Meilleur k : 5


In [54]:
# Entrainement du meilleur modèle et score sur le jeu de test
knn = neighbors.KNeighborsRegressor(n_neighbors=kBest)
knn.fit(X_train_std, y_train)
y_pred = knn.predict(X_test_std)
print ("Erreur sur le jeu de test : MSE = %.2f" % metrics.mean_squared_error(y_test, y_pred))

Erreur sur le jeu de test : MSE = 11.43


In [55]:
# Méthode naive par la moyenne pour comparer
from sklearn import dummy
dum = dummy.DummyRegressor(strategy='mean')
# Entraînement
dum.fit(X_train_std, y_train)
# Prédiction sur le jeu de test
y_pred_dum = dum.predict(X_test_std)
# Evaluate
print ("Méthode naive par la moyenne : MSE = %.2f" % metrics.mean_squared_error(y_test, y_pred_dum))

Méthode naive par la moyenne : MSE = 81.80


Notre modèle est bien meilleur que le modèle naif

In [56]:
# Recherche des scores nulls
X = dfScoreNull.drop(col_score, axis=1)

# Standardisation
std_scale = preprocessing.StandardScaler().fit(X)
X_scaled = std_scale.transform(X)

yScoreNull = knn.predict(X_scaled)

# Remplacement des nulls
dfScoreNull.loc[:,'nutrition_score_fr_100g'] = yScoreNull

my_data_nut = dfScoreNotNull.append(dfScoreNull)
my_data_nut.shape

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(10000, 45)

## Recherche des grades manquants par classification knn

In [57]:
# Séparation des valeurs nulles et non nulles
dfScoreNull = my_data_nut[pd.isnull(my_data_nut.nutrition_grade_fr)]
print('Il y a', dfScoreNull.shape[0], 'valeurs nulles')
dfScoreNotNull = my_data_nut[pd.notnull(my_data_nut.nutrition_grade_fr)]
print('Il y a', dfScoreNotNull.shape[0], 'valeurs non nulles')

Il y a 1679 valeurs nulles
Il y a 8321 valeurs non nulles


In [58]:
# On sépare les colonnes de scores sur les valeurs non nulles
X = dfScoreNotNull.drop(col_score, axis=1)
y = dfScoreNotNull.nutrition_grade_fr

# Transformation des grades en valeurs de 1 à 5, correspondant à (['a', 'b', 'c', 'd', 'e'])
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
label_encoder = enc.fit(y)
y = label_encoder.transform(y)
label_encoder.classes_

# 30% des données dans le jeu de test
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3)

# Standardisation
std_scale = preprocessing.StandardScaler().fit(X_train)
X_train_std = std_scale.transform(X_train)
X_test_std = std_scale.transform(X_test)

In [59]:
param_grid = {'n_neighbors':[3, 5, 7, 9, 11, 13, 15]}
mse = 'neg_mean_squared_error'
gs = model_selection.GridSearchCV(neighbors.KNeighborsClassifier(), param_grid, scoring=mse, cv = 5)                                 

gs.fit(X_train_std, y_train)
me = []
err = []
for mean, std, params in zip(gs.cv_results_['mean_test_score'], # score moyen
                             gs.cv_results_['std_test_score'], # écart-type du score
                             gs.cv_results_['params'] # valeur de l'hyperparamètre
                            ):
    print ("\t%s = %0.3f (+/-%0.03f) for %r" % ('MSE', -mean, std * 2, params))
    me.append(mean)
    err.append(2*std)
kBest = gs.best_params_['n_neighbors']
print ('Meilleur k : %r' % kBest)

	MSE = 0.623 (+/-0.128) for {'n_neighbors': 3}
	MSE = 0.598 (+/-0.089) for {'n_neighbors': 5}
	MSE = 0.623 (+/-0.100) for {'n_neighbors': 7}
	MSE = 0.619 (+/-0.079) for {'n_neighbors': 9}
	MSE = 0.649 (+/-0.068) for {'n_neighbors': 11}
	MSE = 0.655 (+/-0.050) for {'n_neighbors': 13}
	MSE = 0.660 (+/-0.040) for {'n_neighbors': 15}
Meilleur k : 5


In [60]:
# Entrainement du meilleur modèle
knn = neighbors.KNeighborsClassifier(n_neighbors=kBest)
knn.fit(X_train_std, y_train)
y_pred = knn.predict(X_test_std)
print ("Erreur sur le jeu de test : MSE = %.2f" % metrics.mean_squared_error(y_test, y_pred))

Erreur sur le jeu de test : MSE = 0.48


In [61]:
# Méthode naive par la classe la plus fréquente pour comparer
dum = dummy.DummyClassifier(strategy='most_frequent')
# Entraînement
dum.fit(X_train_std, y_train)
# Prédiction sur le jeu de test
y_pred_dum = dum.predict(X_test_std)
# Evaluate
print ("Méthode naive : MSE = %.2f" % metrics.mean_squared_error(y_test, y_pred_dum))

Méthode naive par la moyenne : MSE = 2.54


Notre modèle est bien meilleur que le modèle naif

In [62]:
# Recherche des scores nulls
X = dfScoreNull.drop(col_score, axis=1)

# Standardisation
std_scale = preprocessing.StandardScaler().fit(X)
X_scaled = std_scale.transform(X)

yScoreNull = knn.predict(X_scaled)

#On remet les grades
yScoreNull = label_encoder.inverse_transform(yScoreNull)

# Remplacement des nulls
dfScoreNull.loc[:,'nutrition_grade_fr'] = yScoreNull

my_data_nut = dfScoreNotNull.append(dfScoreNull)
my_data_nut.shape

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(10000, 45)

# Export des résultats

In [63]:
my_data_nut.to_csv('clean_products_full.csv', sep="\t")